# Análisis de derechos de agua (Parte I)

## Descarga de datos desde www.dga.cl

Es necesario un poco de relato respecto de este proceso para entender por qué lo hice de esta manera y no algo más simple.
Obviamente lo primero que intenté fue leer los archivos Excel, publicados por la DGA, usando Pandas, pero esto arrojaba error 500, pese a que navegando usando cualquier browser se pueden descargar, por ello traté también con urllib, pero también me fue mal, por lo que decidí usar selenium, y simular la navegación.

El problema de esto es que requiere un setup incial que las otras alternativas no tienen, obviamente lo primero es instalar selenium, esto lo podemos hacer con un simple (cabe mencionar que estoy usando OSX Mojave 10.14, por lo que algunas instrucciones serán distintas para Windows, actualmente no tengo ningún PC con ese OS, por lo que en cuanto tenga acceso actualizaré estas instrucciones para ese OS)

``` pip install selenium ```


Luego es necesario instalar el driver del browser que vamos a usar para navegar por el sitio
en mi caso usé gecko de Firefox. Para esta instalación seguí la siguiente guía https://www.kenst.com/2016/12/installing-marionette-firefoxdriver-on-mac-osx/ pero que es escencialmente instalar vía ***brew*** con el siguiente comando:

```brew install geckodriver```

Con eso ya estamos en condiciones de descargar los archivos, al menos en OSX :P

Ahora algunos puntos sobre la web donde se encuentran los archvivos. Lo primero es que existen un repositorio de archvios que se actualiza con cierta periodicidad en http://www.dga.cl/productosyservicios/derechos_historicos/Paginas/default.aspx y este será el que utilizaré para trabajar este proyecto. Existe además un buscador en http://derechosdeagua.dga.cl que está implementado en php y que arroja como resultado un xml con extensión xls, que sinceramente no pude trabajar de manera correcta con pandas.

A continuación una imagen del sitio: 

<img src="https://i.ibb.co/8c118VS/Captura-de-Pantalla-2019-10-06-a-la-s-09-12-23.png" alt="Captura-de-Pantalla-2019-10-06-a-la-s-09-12-23" border="0">

Quisiera hacer una mención al hecho que hayan "comprimido" el consolidad nacional y hayan logrado pasar de los 41,5 MB a unos maravillosos a 40,1 MB 🙄

### Descarga del archivo CONSOLIDADO al directorio de trabajo
Las siguientes instrucciones permiten descargar el archivo de derechos de agua de todas las regiones consolidado, y que esta descarga quede en nuestro actual directorio de trabajo para que sea mas fácil trabajar con el posteriormente

In [16]:
# Primero vamos a realizar algunas configuraciones necesarias para que posteriormente la descarga del archivo quede
# en el actual directorio de trabajo
import os
from selenium import webdriver

cwd = os.getcwd() #fijamos el actual directorio de trabajo

# creamos un nuevo perfil de Firefox, para definir que las descargas sean al actial directorio de trabajo
profile = webdriver.FirefoxProfile() 
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.dir", cwd)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")

# Luego cargar las librerias que vamos a usar para la navegación y descarga
from selenium.webdriver.firefox.options import Options

# Generamos el setup para que navegar sin cabeceras
opts = Options()
opts.headless = True
assert opts.headless
browser = webdriver.Firefox(options=opts, firefox_profile=profile)

# Obtenemos la información del sitio
browser.get('http://www.dga.cl/productosyservicios/derechos_historicos/Paginas/default.aspx')

# Clickeamos en el link correspondiente
browser.find_element_by_link_text('Consolidado nacional').click()

### Descompresión del archivo CONSOLIDADO y carga en Pandas
El siguiente código permite descomprimir el archivo descargado, que como ya mencionamos está comprimido en ZIP, aunque con una particularidad, al descargar el archivo con Selenium, a la extensión del archivo se le agregó una extensión **'.part'**, bastante extraña, como si nunca quedas descargada definitivamente, por lo que si alguien sabe como mejorar este aspecto se agradece, por el momento yo tomé el camino largo de cambiar la extensión vía código. 

In [8]:
# Primero vamos a crear un loop que recorra el actual directorio, usando la librería 'os'

files = [] # creamos la lista de archivos donde se almacenará nuestra actual descarga

for r, d, f in os.walk(cwd): # r=root, d=directorios, f = archivos
    for file in f:
        if '.part' in file:
            files.append(os.path.join(r, file))



files[0]

'/Users/josemora/Documents/derechos/yGsqFVcU.zip.part'

In [9]:
# Luego usando la misma librería vamos a renombrar el archivo
thisFile = files[0]
base = os.path.splitext(thisFile)[0]
os.rename(thisFile, base + "")



In [4]:
import os
cwd = os.getcwd() #fijamos el actual directorio de trabajo

In [ ]:
# lectura del archivo Excel descargado y carga en Pandas
import pandas as pd
data = pd.read_excel('derechos.xlsx') 